# Two Tower Neural Networks Recommendation Model

In [129]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import pandas as pd
import numpy as np
import itertools

### Loading Dataset

In [353]:
data = pd.read_csv("./ratings.csv")

In [354]:
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [355]:
data['userId'] = data['userId'] - 1
data['movieId'] = data['movieId'] - 1

In [356]:
data.drop_duplicates(inplace=True)

In [357]:
data.dropna(inplace=True)

In [358]:
def fillID(df, col):
    m = dict({-1:-1})
    new_col = []
    for value in df[col].values:
        if value in m:
            new_col.append(m[value])
        else:
            new_index = max(m.values()) + 1
            m[value] = new_index
            new_col.append(new_index)
    df[col+"Index"] = new_col
    return

In [359]:
fillID(data, 'userId')
fillID(data, 'movieId')

In [360]:
def normalize(data):
    """
    Normalize the data to the range [0, 1].
    
    Parameters:
    data (numpy array): Input data to be normalized.
    
    Returns:
    normalized_data (numpy array): Normalized data.
    """
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data

### Converting Continous Rating to Class

In [361]:
# data['rating] = normalize(data['rating'].values)
def get_class(x):
    if x < 4.0:
        return 0
    return 1
data['rating'] = data['rating'].apply(lambda x: get_class(x))

In [362]:
data['rating'].value_counts()

0    52256
1    48580
Name: rating, dtype: int64

### Train/Test Splitting

In [363]:
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8 # 80% Train, 20% Test
train = data[msk].copy()
validation = data[~msk].copy()

In [364]:
len(data), len(train), len(validation)

(100836, 80450, 20386)

In [365]:
train.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'userIdIndex',
       'movieIdIndex'],
      dtype='object')

In [366]:
num_users = data['userIdIndex'].nunique()
num_movies = data['movieIdIndex'].nunique()

In [367]:
print(num_users, num_movies)

610 9724


In [368]:
train.head()

,userId,movieId,rating,timestamp,userIdIndex,movieIdIndex
0,0,0,1,964982703,0,0
1,0,2,1,964981247,0,1
2,0,5,1,964982224,0,2
3,0,46,1,964983815,0,3
6,0,100,1,964980868,0,6


### Dataset and Dataloader

In [415]:
batch_size = 5000

train_features = torch.LongTensor(train[['userIdIndex', 'movieIdIndex']].values)
train_target = torch.Tensor(train[['rating']].values).float()

train_ds = TensorDataset(train_features, train_target)
dl_train = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)

val_features = torch.LongTensor(validation[['userIdIndex', 'movieIdIndex']].values)
val_target = torch.Tensor(validation[['rating']].values).float()

val_ds = TensorDataset(val_features, val_target)
dl_val = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, drop_last=True)

In [416]:
xb, yb = next(iter(dl_train))
print(xb)
print(yb)

tensor([[  26, 1768],
        [  80,  621],
        [ 232, 1932],
        ...,
        [ 130, 2808],
        [ 232, 5744],
        [ 520,  523]])
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])


### Two Tower Model

![alt text](model.png "Two Tower Model")

### Tower

In [371]:
# Towers Model
class Tower(nn.Module):
    def __init__(self, num_ids, emb_size=64):
        super(Tower, self).__init__()
        self.embedding = nn.Embedding(num_ids, emb_size)
        self.embedding.weight.data.uniform_(0, 0.05)
        
        # FC layers
        self.fc1 = nn.Linear(emb_size, emb_size//2)
        self.fc2 = nn.Linear(emb_size//2, emb_size//4)
    
    def forward(self, x):
        emb = self.embedding(x)
        x = nn.functional.relu(self.fc1(emb))
        x = nn.functional.relu(self.fc2(x))
        return x

In [372]:
t1 = Tower(10, 20)
t1(torch.Tensor([5]).long())

tensor([[0.0000, 0.4029, 0.3279, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)

### Common Deep Neural Network

In [377]:
class DNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size * 2)
        self.fc2 = nn.Linear(input_size * 2, output_size)
    
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [378]:
dnn = DNN(5, 1)
out = dnn(torch.Tensor([1.0,2.0,3.0,4.0,5.0]))
out

tensor([0.4133], grad_fn=<SigmoidBackward0>)

In [380]:
F.binary_cross_entropy(out, torch.Tensor([1.0]))

tensor(0.8835, grad_fn=<BinaryCrossEntropyBackward0>)

### Training Loop

In [438]:
def test(tower1, tower2, dnn):
    tower1.eval()
    tower2.eval()
    dnn.eval()
    
    total_loss = []
    for indices, ratings in dl_val:
        ratings = ratings.squeeze(1)
        users, items = indices[:,0], indices[:,1]
        # Compute model output
        out1 = tower1(users)
        out2 = tower2(items)
        output = dnn(torch.cat([out1, out2], dim=1))
        loss = F.binary_cross_entropy(output, ratings.unsqueeze(1))
        total_loss.append(loss.item())
    
    print(f"Test Loss: {sum(total_loss) / len(total_loss)}")
    
    
def trainer(tower1, tower2, dnn, num_epochs, optimizer):
    # Set model in Training mode
    tower1.train()
    tower2.train()
    dnn.train()
    
    # Start training loop
    for epoch in range(num_epochs):
        total_loss = []
        for indices, ratings in dl_train:
            ratings = ratings.squeeze(1)
            users, items = indices[:,0], indices[:,1]
            # Compute model output
            out1 = tower1(users)
            out2 = tower2(items)
            out = torch.cat([out1, out2], dim=1)
            output = dnn(out)
            # Compute Loss
            loss = F.binary_cross_entropy(output, ratings.unsqueeze(1))

            # Update model weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss.append(loss.item())
        
        # Print Error
        print(f"{epoch}/{num_epochs} - Loss: {sum(total_loss)/len(total_loss)}")
        
        # Test model at regular intervals
        if epoch % 10 == 0:
            # Test model
            test(tower1, tower2, dnn)
        

### Initialize Model

In [439]:
user_tower = Tower(610) # num users
item_tower = Tower(9724) # num items
dnn = DNN(32, 1)

In [440]:
# Init optimizer
parameters = list(user_tower.parameters()) + list(item_tower.parameters()) + list(dnn.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

### Training

In [449]:
trainer(user_tower, item_tower, dnn, 10, optimizer)

0/10 - Loss: 0.46899214211632223
Test Loss: 0.4599958937615156
1/10 - Loss: 0.46612585467450757
2/10 - Loss: 0.46353044755318584
3/10 - Loss: 0.4605325986357296
4/10 - Loss: 0.4624222394298105
5/10 - Loss: 0.4555685485110563
6/10 - Loss: 0.4533233081593233
7/10 - Loss: 0.4509568652685951
8/10 - Loss: 0.44538164840025063
9/10 - Loss: 0.4409148184692158


In [450]:
test(user_tower, item_tower, dnn)

Test Loss: 0.4321808349341154


### Sanity Checks

In [451]:
indices, ratings = next(iter(dl_val))
ratings = ratings.squeeze(1)
users, items = indices[:,0], indices[:,1]
# Compute model output
out1 = user_tower(users)
out2 = item_tower(items)
output = dnn(torch.cat([out1, out2], dim=1))
loss = F.mse_loss(output, ratings.unsqueeze(1))
print("Loss: ",loss.item())

Loss:  0.14655284583568573


In [452]:
ratings.unsqueeze(1)

tensor([[0.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]])

In [453]:
output

tensor([[7.5827e-01],
        [5.9585e-08],
        [9.0548e-01],
        ...,
        [8.4901e-01],
        [9.6559e-01],
        [5.8793e-01]], grad_fn=<SigmoidBackward0>)

### Compute Accuracy for Batch

In [454]:
threshold = 0.5

# Convert to class labels
class_labels = (output > threshold).int()

In [455]:
class_labels

tensor([[1],
        [0],
        [1],
        ...,
        [1],
        [1],
        [1]], dtype=torch.int32)

In [456]:
correct_predictions = (class_labels == ratings.unsqueeze(1)).sum().item()
total_samples = ratings.size(0)
accuracy = correct_predictions / total_samples * 100

print("Accuracy:", accuracy, "%")

Accuracy: 77.52 %
